In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset


In [4]:
# Load the SQuAD dataset
squad_dataset = load_dataset("squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering ,TrainingArguments, Trainer
import numpy as np
import pandas as pd
import json


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [6]:
def preprocess_function(examples):
    questions = examples["question"]
    contexts = examples["context"]
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation=True,
        padding="max_length",
        return_offsets_mapping=True,
    )
    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []

    for i in range(len(questions)):
        if "answers" in examples and len(examples["answers"][i]["answer_start"]) > 0:
            answer_start = examples["answers"][i]["answer_start"][0]
            answer_text = examples["answers"][i]["text"][0]
            start_char = answer_start
            end_char = start_char + len(answer_text)
            start_pos = None
            end_pos = None

            # Find token positions for start and end
            for j, (start, end) in enumerate(offset_mapping[i]):
                if start_pos is None and start_char >= start and start_char < end:
                    start_pos = j
                if end_pos is None and end_char > start and end_char <= end:
                    end_pos = j

            start_positions.append(start_pos if start_pos is not None else 0)
            end_positions.append(end_pos if end_pos is not None else 0)
        else:
            start_positions.append(0)
            end_positions.append(0)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [11]:
# Preprocess the training and validation datasets
train_dataset = squad_dataset["train"].map(
    preprocess_function,
    batched=True,
    remove_columns=["id", "title", "context", "question", "answers"]
)

validation_dataset = squad_dataset["validation"].map(
    preprocess_function,
    batched=True,
    remove_columns=["id", "title", "context", "question", "answers"]
)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [7]:
# Use a pre-trained model and tokenizer
model_name = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [12]:
# Define training arguments
training_args = TrainingArguments(
output_dir="./results",
evaluation_strategy="epoch",
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
learning_rate=5e-5,
num_train_epochs=2,
weight_decay=0.01,
save_total_limit=1
)
# Create a Trainer instance
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=validation_dataset
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.962900,1.354435
2,0.628400,1.426487


TrainOutput(global_step=10950, training_loss=0.8297689777531035, metrics={'train_runtime': 7114.2057, 'train_samples_per_second': 24.627, 'train_steps_per_second': 1.539, 'total_flos': 1.7167621364554752e+16, 'train_loss': 0.8297689777531035, 'epoch': 2.0})

In [14]:
result = trainer.evaluate()

In [15]:
context = "Paris is the capital and most populous city of France."
question = "What is the capital of France?"

In [16]:
inputs = tokenizer(
    question,
    context,
    return_tensors="pt",
    max_length=384,
    truncation=True,
    padding="max_length",
    return_offsets_mapping=True,
)

In [17]:
offset_mapping = inputs.pop("offset_mapping")

In [21]:
import torch

# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Move inputs to the GPU
inputs = {k: v.to(device) for k, v in inputs.items()}

# Get predictions from the model
outputs = model(**inputs)

In [22]:
# Extract start and end logits
start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Find start and end indices with additional validation
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)

In [23]:
sep_index = inputs["input_ids"][0].tolist().index(tokenizer.sep_token_id)

# Ensure the start and end indices point to the context section
start_index = max(start_index, sep_index + 1)
end_index = max(end_index, sep_index + 1)

# Ensure start_index isn't greater than end_index
if start_index > end_index:
    start_index, end_index = min(start_index, end_index), max(start_index, end_index)

# Extract the predicted answer from the context
if start_index <= end_index:
    answer_tokens = inputs["input_ids"][0][start_index:end_index + 1]
    predicted_answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
else:
    predicted_answer = ""

print("Predicted Answer:", predicted_answer)

Predicted Answer: Paris
